In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
data = np.load("train.npy")

FileNotFoundError: [Errno 2] No such file or directory: 'train.npy'

In [ ]:
X = []
Y = []
i = 0
for elem in data:
    if i % 1000 == 0:
        print(i)
    X.append(tf.resize(elem[0], output_shape=(40,40)))
    Y.append(elem[1])
    i += 1

In [12]:
np.save('X', X)

In [13]:
np.save('Y', Y)

In [29]:
X = np.array(X)
Y = np.array(Y)

In [2]:
import numpy as np
import pandas as pd

X = np.load("X.npy")
Y = np.load("Y.npy")

X.resize((166708,40,40,1))

labels = np.unique(Y)
labels = labels.tolist()

for i in range(len(Y)):
    Y[i] = labels.index(Y[i])

X = X.astype(np.float16)
Y = Y.astype(np.int16)

## Data augmentation


In [31]:
X = np.load("X.npy")
Y = np.load("Y.npy")

In [7]:
from skimage.filters import threshold_adaptive
from skimage.morphology import skeletonize
from skimage.restoration import denoise_bilateral
plt.imshow(denoise_bilateral(X[0], multichannel=False), 'gray')
plt.figure()
plt.imshow(X[0], 'gray')

NameError: name 'X' is not defined

In [10]:
from tqdm import tqdm
from skimage.transform import rotate
X_denoise_test = []

for elem in tqdm(test_set):
    #X_denoise_test.append(rotate(denoise_bilateral(elem,multichannel=False),
                     #np.random.randint(low = -7, high=7) , resize=False))
    X_denoise_test.append(denoise_bilateral(elem,multichannel=False))

100%|██████████| 41428/41428 [01:12<00:00, 570.84it/s]  | 46/41428 [00:00<01:31, 453.04it/s]


In [62]:
np.save('X_denoise_rotate', X_denoise_rotate)

In [5]:
X_aug = []
Y_aug = []

for epoch in range(2):
    
    for i in range(len(X)):
        if i % 10000 == 0:
            print(epoch, i)
        temp = X[i]
        temp = noise_filter(temp, disk(np.random.random()))

        X_aug.append(rotate(X[i], np.random.randint(low = -30, high=30) , resize=False))
        X_aug.append(rotate(temp, np.random.randint(low = -30, high=30) , resize=False))
        Y_aug.append(Y[i])
        Y_aug.append(Y[i])

0 0


/usr/local/lib/python3.5/dist-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


0 10000
0 20000
0 30000
0 40000
0 50000
0 60000
0 70000
0 80000
0 90000
0 100000
0 110000
0 120000
0 130000
0 140000
0 150000
0 160000
1 0
1 10000
1 20000
1 30000
1 40000
1 50000
1 60000
1 70000
1 80000
1 90000
1 100000
1 110000
1 120000
1 130000
1 140000
1 150000
1 160000


In [12]:
size = int(len(X_aug)*7/10)
np.save('X_aug', X_aug[:size])
np.save('Y_aug', Y_aug)

In [2]:
X_denoise_rotate = np.load("X_denoise_rotate.npy")

In [13]:
X_denoise_test = np.array(X_denoise_test)
X_denoise_test.resize((len(X_denoise_test),40,40,1))

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=42)

In [24]:
from sklearn.model_selection import train_test_split

X_train_aug, X_test_aug, y_train_aug, y_test_aug = train_test_split(X_aug, Y_aug[:int(len(Y_aug) * 7 / 10)], test_size=0.3, random_state=42)

In [6]:
X_flat = []

for elem in X:
    X_flat.append(elem.flatten())
X_flat = np.array(X_flat)

In [60]:
import xgboost as xgb
from sklearn.grid_search import GridSearchCV

model = xgb.XGBClassifier(max_depth=7, silent=False)

model.fit(xgb_pred_clas, y_test)

y_pred = model.predict(X_test)
# evaluate predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

ValueError: Input numpy.ndarray must be 2 dimensional

In [5]:
import densenet
from keras.callbacks import TensorBoard
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import keras

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.advanced_activations import PReLU
from keras.callbacks import ModelCheckpoint
from keras.layers import ZeroPadding2D

from keras.callbacks import TensorBoard


Y_train = np_utils.to_categorical(y_train, 500)
Y_test = np_utils.to_categorical(y_test, 500)


# 'th' dim-ordering or 'tf' dim-ordering
image_dim = (40, 40, 1)

model = densenet.create_dense_net(nb_classes=500, img_dim=image_dim, depth=40, growth_rate=12, 
                                   bottleneck=True, reduction=0.5)


model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

tensorbord = TensorBoard(log_dir='/home/axcel/logs', histogram_freq=0,
                                         write_graph=True, write_images=False)

model_saver = ModelCheckpoint("weights.{epoch:02d}-{val_acc:.6f}.hdf5",
                                              monitor='val_acc', verbose=0,
                                              save_best_only=True, save_weights_only=False,
                                              mode='auto')



datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2)
datagen.fit(X_train)




model.fit_generator(datagen.flow(X_train, Y_train, batch_size=64),
                    samples_per_epoch=len(X_train), nb_epoch=1, verbose=2,
                    validation_data=(X_test, Y_test), callbacks=[tensorbord, model_saver])

#model.fit(X_train, Y_train, batch_size=64, nb_epoch=3,
          #verbose=2, validation_data=(X_test, Y_test), callbacks=[tensorbord, model_saver])

Bottleneck DenseNet-BC-40-12 with 0.5 compression created.
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 40, 40, 1)     0                                            
____________________________________________________________________________________________________
initial_conv2D (Convolution2D)   (None, 40, 40, 24)    216         input_2[0][0]                    
____________________________________________________________________________________________________
batchnormalization_40 (BatchNorm (None, 40, 40, 24)    48          initial_conv2D[0][0]             
____________________________________________________________________________________________________
activation_40 (Activation)       (None, 40, 40, 24)    0           batchnormalization_40[0][0]      
________________________________

In [94]:
ans = model.predict(xgb_pred_clas)

In [95]:
ans = ans.astype(np.int32)

In [96]:
ans

array([ 12,  52,  92, ..., 373, 240, 488], dtype=int32)

In [6]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import keras

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.advanced_activations import PReLU
from keras.callbacks import ModelCheckpoint
from keras.layers import ZeroPadding2D

from keras.callbacks import TensorBoard

batch_size = 512
nb_classes = 500
nb_epoch = 10

# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

input_shape = (40,40,1)

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

prelu = keras.layers.advanced_activations.PReLU(init='zero', weights=None)

model = Sequential()


model.add(ZeroPadding2D((1,1),input_shape=(40,40,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(BatchNormalization())

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(BatchNormalization())

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(BatchNormalization())

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(BatchNormalization())

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))

model.add(BatchNormalization())

model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(500, activation='softmax'))



model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

tensorbord = TensorBoard(log_dir='/home/axcel/logs', histogram_freq=0,
                                         write_graph=True, write_images=False)

model_saver = ModelCheckpoint("weights.{epoch:02d}-{val_acc:.6f}.hdf5",
                                              monitor='val_acc', verbose=0,
                                              save_best_only=True, save_weights_only=False,
                                              mode='auto')



#datagen = ImageDataGenerator(
    #featurewise_center=False,
    #featurewise_std_normalization=False,
    #rotation_range=10,
    #width_shift_range=0.2,
    #height_shift_range=0.2)

#datagen.fit(X_train)




#model.fit_generator(datagen.flow(X_train, Y_train, batch_size=64),
                    #samples_per_epoch=len(X_train), nb_epoch=1, verbose=2,
                    #validation_data=(X_test, Y_test), callbacks=[tensorbord, model_saver])

model.fit(X_train, Y_train, batch_size=64, nb_epoch=3,
          verbose=2, validation_data=(X_test, Y_test), callbacks=[tensorbord, model_saver])
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
zeropadding2d_33 (ZeroPadding2D) (None, 42, 42, 1)     0           zeropadding2d_input_3[0][0]      
____________________________________________________________________________________________________
convolution2d_33 (Convolution2D) (None, 40, 40, 64)    640         zeropadding2d_33[0][0]           
____________________________________________________________________________________________________
zeropadding2d_34 (ZeroPadding2D) (None, 42, 42, 64)    0           convolution2d_33[0][0]           
____________________________________________________________________________________________________
convolution2d_34 (Convolution2D) (None, 40, 40, 64)    36928       zeropadding2d_34[0][0]           
___________________________________________________________________________________________

In [18]:
X_aug = np.array(X_aug[:size], copy=False)
X_aug.resize((len(X_aug),40,40,1))

labels = np.unique(Y_aug)
labels = labels.tolist()

for i in range(len(Y_aug)):
    Y_aug[i] = labels.index(Y_aug[i])

X_aug = X_aug.astype(np.float16)
Y_aug = np.array(Y_aug).astype(np.int16)

In [4]:
from keras.models import load_model


model = load_model("./Models Architecture/densenet.hdf5")

Using TensorFlow backend.


In [ ]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import keras

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.advanced_activations import PReLU
from keras.callbacks import ModelCheckpoint
from keras.layers import ZeroPadding2D

from keras.callbacks import TensorBoard

batch_size = 400
nb_classes = 500
nb_epoch = 10

# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

input_shape = (40,40,1)

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

tensorbord = TensorBoard(log_dir='/home/axcel/logs', histogram_freq=0,
                                         write_graph=True, write_images=True)

model_saver = ModelCheckpoint("weights.{epoch:02d}-{val_acc:.6f}.hdf5",
                                              monitor='val_acc', verbose=0,
                                              save_best_only=True, save_weights_only=False,
                                              mode='auto')


datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2)

datagen.fit(X_train)

model.fit_generator(datagen.flow(X_train, Y_train, batch_size=64),
                    samples_per_epoch=len(X_train), nb_epoch=5, verbose=2,
                    validation_data=(X_test, Y_test), callbacks=[tensorbord, model_saver] )
#model.fit(X_train, Y_train, batch_size=64, nb_epoch=50,
          #verbose=2, validation_data=(X_test, Y_test), callbacks=[tensorbord, model_saver])
#print('Test score:', score[0])
#print('Test accuracy:', score[1])

Instructions for updating:
Please switch to tf.summary.merge_all.
Instructions for updating:
Please switch to tf.summary.FileWriter. The interface and behavior is the same; this is just a rename.
Epoch 1/5
1382s - loss: 0.6515 - acc: 0.9428 - val_loss: 0.3394 - val_acc: 0.9102
Epoch 2/5
1373s - loss: 0.6477 - acc: 0.9419 - val_loss: 1.7550 - val_acc: 0.5823
Epoch 3/5
1374s - loss: 0.6405 - acc: 0.9426 - val_loss: 0.2663 - val_acc: 0.9331
Epoch 4/5
1374s - loss: 0.6346 - acc: 0.9443 - val_loss: 0.2249 - val_acc: 0.9411
Epoch 5/5


In [13]:
model.save("VGG9881")

In [1]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import numpy as np

Using TensorFlow backend.


In [7]:
from keras.models import load_model

#model = load_model("./Models Architecture/mnist_cnn_architecture_first_sub")

In [16]:
test = np.load('test.npy')

In [17]:
import skimage.transform

test_set = []
i = 0
for elem in test:
    if i % 1000 == 0:
        print(i)
    test_set.append(skimage.transform.resize(elem, output_shape=(40,40)))
    i += 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000


In [18]:
test_set = np.array(test_set)

In [19]:
test_set.resize((41428,40,40,1))

In [14]:
ans = model.predict_classes(test_set, verbose=0)

In [14]:
from keras.models import load_model
model1 = load_model("./Models Architecture/3 conv with maxpool and batchnorm with 2 dense 20 epoch")
model2 = load_model("./Models Architecture/3 conv with maxpool and batchnorm with 2 dense 20 epoch + 10 augmented")
model3 = load_model("./Models Architecture/4 conv with maxpool and batchnorm")
model4 = load_model("./Models Architecture/4 conv with maxpool and batchnorm V2")
model5 = load_model("./Models Architecture/4 conv with maxpool and batchnorm V2 200 epoch")
model6 = load_model("./Models Architecture/mnist_cnn_architecture_first_sub")
model7 = load_model("./Models Architecture/3 conv with maxpool and batchnorm with 2 dense 20 epoch + 30 denoised and rotated")
model8 = load_model("./Models Architecture/3 conv with maxpool and batchnorm mode1 with 2 dense 50 epoch")
model9 = load_model("./Models Architecture/3 conv with maxpool and batchnorm 2 dense on augmented 50 epoch")
model10 = load_model("./Models Architecture/3 conv with maxpool and batchnorm 2 dense_small on augmented 50 epoch")
model11 = load_model("./Models Architecture/4 conv with maxpool and batchnorm 2 dense on augmented 40 epoch")
model12 = load_model("./Models Architecture/4 conv with maxpool and batchnorm 2 dense sigmoid on augmented 50 epoch")
model13 = load_model("./Models Architecture/conv5.hdf5")
model14 = load_model("./Models Architecture/4convpoolbigdense.hdf5")
model15 = load_model("./Models Architecture/4convbatchmediumdense.hdf5")
model16 = load_model("./Models Architecture/3smallconvprelu.hdf5")
model17 = load_model("./Models Architecture/4convzeropad.hdf5")
model18 = load_model("./Models Architecture/VGG9766.hdf5")
model19 = load_model("VGG9881")

In [85]:
model16 = load_model("./Models Architecture/3smallconvprelu.hdf5")

In [20]:
prediction1 = model1.predict_proba(test_set, verbose=2)
prediction2 = model2.predict_proba(test_set, verbose=2)
prediction3 = model3.predict_proba(test_set, verbose=2)
prediction4 = model4.predict_proba(test_set, verbose=2)
prediction5 = model5.predict_proba(test_set, verbose=2)
prediction6 = model6.predict_proba(test_set, verbose=2)
prediction7 = model7.predict_proba(test_set, verbose=2)
prediction8 = model8.predict_proba(test_set, verbose=2)
prediction9 = model9.predict_proba(test_set, verbose=2)
prediction10 = model10.predict_proba(test_set, verbose=2)
prediction11 = model11.predict_proba(test_set, verbose=2)
prediction12 = model12.predict_proba(test_set, verbose=2)
prediction13 = model13.predict_proba(test_set, verbose=2)
prediction14 = model14.predict_proba(test_set, verbose=2)
prediction15 = model15.predict_proba(test_set, verbose=2)
prediction16 = model16.predict_proba(test_set, verbose=2)
prediction17 = model17.predict_proba(test_set, verbose=2)
prediction18 = model18.predict_proba(test_set, verbose=2)
prediction19 = model19.predict_proba(test_set, verbose=2)

NameError: name 'mdeol19' is not defined

In [21]:
prediction19 = model19.predict_proba(test_set, verbose=2)

In [71]:
prediction1 = model1.predict_proba(X_test, verbose=2)
prediction2 = model2.predict_proba(X_test, verbose=2)
prediction3 = model3.predict_proba(X_test, verbose=2)
prediction4 = model4.predict_proba(X_test, verbose=2)
prediction5 = model5.predict_proba(X_test, verbose=2)
prediction6 = model6.predict_proba(X_test, verbose=2)
prediction7 = model7.predict_proba(X_test, verbose=2)
prediction8 = model8.predict_proba(X_test, verbose=2)
prediction9 = model9.predict_proba(X_test, verbose=2)
prediction10 = model10.predict_proba(X_test, verbose=2)
prediction11 = model11.predict_proba(X_test, verbose=2)
prediction12 = model12.predict_proba(X_test, verbose=2)
prediction13 = model13.predict_proba(X_test, verbose=2)
prediction14 = model14.predict_proba(X_test, verbose=2)
prediction15 = model15.predict_proba(X_test, verbose=2)
prediction16 = model16.predict_proba(X_test, verbose=2)

In [63]:
prediction1 = model1.predict_classes(test_set, verbose=2)
prediction2 = model2.predict_classes(test_set, verbose=2)
prediction3 = model3.predict_classes(test_set, verbose=2)
prediction4 = model4.predict_classes(test_set, verbose=2)
prediction5 = model5.predict_classes(test_set, verbose=2)
prediction6 = model6.predict_classes(test_set, verbose=2)
prediction7 = model7.predict_classes(test_set, verbose=2)
prediction8 = model8.predict_classes(test_set, verbose=2)
prediction9 = model9.predict_classes(test_set, verbose=2)
prediction10 = model10.predict_classes(test_set,verbose=2)
prediction11 = model11.predict_classes(test_set,verbose=2)
prediction12 = model12.predict_classes(test_set,verbose=2)
prediction13 = model13.predict_classes(test_set,verbose=2)
prediction14 = model14.predict_classes(test_set,verbose=2)
prediction15 = model15.predict_classes(test_set,verbose=2)
prediction16 = model16.predict_classes(test_set,verbose=2)

In [ ]:
prediction1 = model1.predict_classes(X_test, verbose=2)
prediction2 = model2.predict_classes(X_test, verbose=2)
prediction3 = model3.predict_classes(X_test, verbose=2)
prediction4 = model4.predict_classes(X_test, verbose=2)
prediction5 = model5.predict_classes(X_test, verbose=2)
prediction6 = model6.predict_classes(X_test, verbose=2)
prediction7 = model7.predict_classes(X_test, verbose=2)
prediction8 = model8.predict_classes(X_test, verbose=2)
prediction9 = model9.predict_classes(X_test, verbose=2)
prediction10 = model10.predict_classes(X_test,verbose=2)
prediction11 = model11.predict_classes(X_test,verbose=2)
prediction12 = model12.predict_classes(X_test,verbose=2)
prediction13 = model13.predict_classes(X_test,verbose=2)
prediction14 = model14.predict_classes(X_test,verbose=2)
prediction15 = model15.predict_classes(X_test,verbose=2)
prediction16 = model16.predict_classes(X_test,verbose=2)

In [64]:
preds_first_level = [prediction1,prediction2,prediction3,prediction4,
                          prediction5,prediction6,prediction7,prediction8,
                          prediction9,prediction10,prediction11,
                          prediction12,prediction13,prediction14,prediction15, prediction16]

In [65]:
for i in range(len(preds_first_level)):
    preds_first_level[i] = preds_first_level[i].reshape((len(preds_first_level[i]),1))

In [66]:
xgb_pred_clas = np.hstack(preds_first_level)

In [43]:
len(prediction1)

25007

In [58]:
xgb_pred_clas[4354]

array([180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180,
       180, 180, 180])

In [96]:
summury_predict = (prediction1 + prediction2 + prediction3 +
                   prediction4 + prediction5 + prediction6 +
                   prediction7 + prediction8 + prediction9 +
                   prediction10 + prediction11 + prediction12 +
                  prediction13 + prediction14 + prediction15 + prediction16)

In [76]:
summury_predict = (best_alph[0]*prediction5 + best_alph[1]*prediction7
                 + best_alph[2]*prediction8 + best_alph[3]*prediction9 +
                   best_alph[4]*prediction10 + best_alph[5]*prediction11 + best_alph[6]*prediction12 +
                  best_alph[7]*prediction13 + best_alph[8]*prediction14 + best_alph[9]*prediction16)

In [22]:
summury_predict = (prediction5 +
                   prediction7 + prediction8 + prediction9 +
                   prediction10 + prediction11 + prediction12 +
                  prediction13 + prediction14 + prediction16 + prediction17 + prediction18 + prediction19)

In [17]:
#harmonic

summury_predict = 10/(1/prediction5 +
                   1/prediction7 + 1/prediction8 + 1/prediction9 +
                   1/prediction10 + 1/prediction11 + 1/prediction12 +
                  1/prediction13 + 1/prediction14 + 1/prediction16)


/home/axcel/.local/lib/python3.5/site-packages/ipykernel/__main__.py:6: RuntimeWarning: divide by zero encountered in true_divide


In [72]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score
alph_coef = [1, 0.7, 0.2]
best_alph = []
max_acc = 0


for i1 in tqdm(alph_coef):
    for i2 in alph_coef:
        for i3 in alph_coef:
            for i4 in alph_coef:
                for i5 in alph_coef:
                    for i6 in alph_coef:
                        for i7 in tqdm(alph_coef):
                            for i8 in alph_coef:
                                for i9 in alph_coef:
                                    for i10 in alph_coef:
                                        summury_predict = (i1*prediction5 + i2*prediction7
                                                           + i3*prediction8 + i4*prediction9 +
                                                           i5*prediction10 + i6*prediction11 +
                                                           i7*prediction12 + i8*prediction13 + i9*prediction14
                                                          + i10*prediction16)
                                        ans = []
                                        for elem in summury_predict:
                                            ans.append(np.argmax(elem))
                                        accuracy = accuracy_score(y_test, ans)
                                        if accuracy > max_acc:
                                            max_acc = accuracy
                                            best_alph = [i1,i2,i3,i4,i5,i6,i7,i8,i9,i10]

100%|██████████| 3/3 [00:15<00:00,  5.06s/it]

100%|██████████| 3/3 [00:14<00:00,  4.93s/it]

100%|██████████| 3/3 [00:14<00:00,  4.94s/it]

100%|██████████| 3/3 [00:14<00:00,  4.93s/it]

100%|██████████| 3/3 [00:14<00:00,  4.91s/it]

100%|██████████| 3/3 [00:14<00:00,  4.93s/it]

100%|██████████| 3/3 [00:14<00:00,  4.97s/it]

100%|██████████| 3/3 [00:14<00:00,  4.96s/it]

100%|██████████| 3/3 [00:14<00:00,  4.96s/it]

100%|██████████| 3/3 [00:14<00:00,  4.96s/it]

100%|██████████| 3/3 [00:14<00:00,  4.96s/it]

100%|██████████| 3/3 [00:14<00:00,  4.97s/it]

100%|██████████| 3/3 [00:14<00:00,  4.91s/it]

100%|██████████| 3/3 [00:14<00:00,  4.88s/it]

100%|██████████| 3/3 [00:14<00:00,  4.92s/it]

100%|██████████| 3/3 [00:14<00:00,  4.91s/it]

100%|██████████| 3/3 [00:14<00:00,  4.91s/it]

100%|██████████| 3/3 [00:14<00:00,  4.91s/it]

100%|██████████| 3/3 [00:14<00:00,  4.93s/it]

100%|██████████| 3/3 [00:14<00:00,  4.88s/it]

100%|██████████| 3/3 [00:14<00:00,  4.88s/it]

100%|████████

In [73]:
max_acc

0.99936017914983799

In [74]:
best_alph

[1, 0.2, 0.2, 0.2, 0.2, 0.2, 1, 0.2, 0.2, 0.2]

In [23]:
ans = []
for elem in summury_predict:
    ans.append(np.argmax(elem))

In [85]:
xgb_ans = ans

In [93]:
xgb_ans

array([-3388, -3132, -2876, ..., -1077, -1861,  -310], dtype=int16)

In [24]:
for i in range(len(ans)):
    ans[i] = labels[ans[i]]

In [25]:
ans = pd.DataFrame(ans)

In [26]:
temp = [i+1 for i in range(len(test_set))]

temp = pd.DataFrame(temp)

In [27]:
result = pd.concat([temp, ans], axis=1)

In [28]:
result.columns = ['Id', 'Category']

In [29]:
result = result.as_matrix()

In [30]:
import numpy
numpy.savetxt("14netVGGfull.csv", result, delimiter=",")